In [1]:
import gpw_queries as gpw
from river_inputs_loader import load_river_inputs
import geometry_helper as gh

# Filter data

In [2]:
data = gpw.get_sites_by_country("india")

In [3]:
data.keys()

Index(['geometry', 'Distance to Waterway (m)', 'Drainage Direction',
       'Elevation', 'Fine Earth Density (kg / m^3)',
       'Height Above Nearest Drainage (m)', 'Landform Type',
       'Nearest Water Type', 'Population - 1 km', 'Population - 10 km',
       'Population - 5 km', 'Slope (degrees)', 'Soil Clay Fraction',
       'Soil Great Group', 'Soil Sand Fraction',
       'Upstream Drainage Area (km^2)', 'area', 'area (km^2)', 'country', 'id',
       'place_name', 'risk'],
      dtype='object')

In [4]:
data.head()

,geometry,Distance to Waterway (m),Drainage Direction,Elevation,Fine Earth Density (kg / m^3),Height Above Nearest Drainage (m),Landform Type,Nearest Water Type,Population - 1 km,Population - 10 km,...,Soil Clay Fraction,Soil Great Group,Soil Sand Fraction,Upstream Drainage Area (km^2),area,area (km^2),country,id,place_name,risk
0,POINT (77.43796 9.94385),1871.306840842308,east,353,143,21.399993896484375,Upper slope (warm),reservoir,1861,236418,...,0.28,0,0.5,0.025265906006097794,1994.3882920761755,0.0019943882920761755,India,8f603571132eb30,"Veerapandi - Seeppalakottai Road, Theni, Theni...",0.1713068804547574
1,POINT (74.99707 23.35251),739.0126046637928,west,476,154,8.100006103515625,Lower slope (flat),river,1056,287216,...,0.46,0,0.26,0.015720102936029434,5852.5284566601495,0.00585252845666015,India,8f42c92d04d554c,"Junwaniya, Ratlam Nagar Tahsil, Ratlam, Madhya...",0.14673485075254383
2,POINT (74.94934 28.31923),1378.338405725306,northwest,287,147,2.899993896484375,Lower slope (flat),water,2759,108979,...,0.25,0,0.5,0.00754183903336525,1997.3005165642267,0.0019973005165642268,India,8f3da4446642a21,"Gajsar, Churu Tehsil, Churu, Rajasthan, 331001...",0.20297400945017752
3,POINT (70.21870 21.79124),508.62615164536084,southeast,54,159,0,Lower slope (flat),river,3848,120337,...,0.35,0,0.32,4.847708702087402,10409.847647716339,0.010409847647716339,India,8f42c2146d43646,"Upleta Taluka, Rajkot, Gujarat, India",0.24046623006271006
4,POINT (73.48358 22.49636),210.26582578754727,northwest,115,159,4,Lower slope (flat),river,6343,206876,...,0.32,1,0.43,0.01581799052655697,16298.18846145113,0.01629818846145113,India,8f42cab439852a4,"Halol Bypass SH-05, Halol, Halol Taluka, Panch...",0.3084192893633979


In [5]:
data['Distance to Waterway (m)']

0       1871.306840842308
1       739.0126046637928
2       1378.338405725306
3      508.62615164536084
4      210.26582578754727
              ...        
685    336.87317224562213
686    295.26655669173886
687    3170.4405343611684
688    246.84113671488052
689    1105.2848079475004
Name: Distance to Waterway (m), Length: 690, dtype: object

In [6]:
filtered_by_water_proximity = data[data['Distance to Waterway (m)'].astype('float32') <= 500.0]
filtered_by_watertype = filtered_by_water_proximity[(filtered_by_water_proximity['Nearest Water Type'].isin(['water', 'stream', 'river', 'coastline', 'drain', 'canal']))]
filtered = filtered_by_watertype[filtered_by_watertype['Distance to Waterway (m)'].astype('float32') != -1]
print(len(filtered))

204


In [7]:
filtered = filtered.sort_values(['risk', 'Distance to Waterway (m)','area (km^2)'], ascending=[False, True, True])


# Enhanced GPW with other datasets

In [8]:
river_inputs_gdf = load_river_inputs("river_inputs/PlasticRiverInputs.shp")

In [9]:
cross_df = gh.ckd_nearest_point_to_point(filtered, river_inputs_gdf)
cross_df = cross_df[cross_df['dist'].astype('float32') <= 0.5]
cross_df = cross_df.sort_values(["m_out_max"], ascending=False)

In [10]:
print(len(cross_df))
cross_df.head()

79


,geometry,Distance to Waterway (m),Drainage Direction,Elevation,Fine Earth Density (kg / m^3),Height Above Nearest Drainage (m),Landform Type,Nearest Water Type,Population - 1 km,Population - 10 km,...,m_out_jul,m_out_aug,m_out_sep,m_out_oct,m_out_nov,m_out_dec,area,mpw,m_out_max,dist
187,POINT (88.33540 22.66677),442.1206350423517,south,5,127,0,Valley,river,3329,4.137416e+06,...,4.045805e+08,1.246114e+09,1.079263e+09,3.470708e+08,3.217228e+07,5.919650e+06,2.972130e+10,98736400.0,1.246114e+09,0.430054
13,POINT (73.11779 19.25087),101.34313832744114,west,3,147,3.299999952316284,Lower slope (flat),river,17824,2.026968e+06,...,2.362883e+08,2.154789e+08,1.010264e+08,7.588453e+06,2.041934e+05,2.973824e+04,4.555700e+09,14350400.0,2.362883e+08,0.208528
197,POINT (73.12484 19.27815),332.34778015490525,northwest,6,148,8.100000381469727,Lower slope (flat),water,3784,1.813954e+06,...,2.362883e+08,2.154789e+08,1.010264e+08,7.588453e+06,2.041934e+05,2.973824e+04,4.555700e+09,14350400.0,2.362883e+08,0.212544
116,POINT (72.82799 19.23437),466.33569222405686,southeast,6,102,3.700000047683716,Lower slope (warm),river,40916,3.896986e+06,...,4.729223e+07,2.623717e+07,1.381968e+07,1.328835e+06,1.849046e+05,6.202310e+04,5.590000e+07,3112740.0,4.729223e+07,0.003343
29,POINT (72.93513 19.06160),208.91423138532338,south,6,144,7,Lower slope (flat),water,76615,5.339669e+06,...,4.163358e+07,2.418122e+07,1.213029e+07,1.041626e+06,1.341036e+05,4.432832e+04,5.920000e+07,2928760.0,4.163358e+07,0.027380
